In [100]:
# Importing required libraries
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd
import random
import copy

In [101]:
# reading the file
df = pd.read_json("train.json")
df_ingredients = df['ingredients'].tolist()
index = df.index
number_of_rows = len(index)

In [102]:
# creating the nature basket
NB = []
for ingredient_list in df_ingredients:
  for ingredient in ingredient_list:
    if ingredient in NB:
      pass
    else:
      NB.append(ingredient)

In [103]:
# creating the kitchen basket
KB = random.sample(NB, 50)

# taking random recipes from the given dataset
recipes = df.sample(n = 500)

# all components of primordial cuisine are extracted

In [104]:
# picking up a random recipe for modification

def is_same(to_check):
  for index, row in recipes.iterrows():
    if row['ingredients']==to_check:
      return True
  return False

def replace_ingredient():
  modification_recipe = recipes.sample()
  modification_recipe_id = int(modification_recipe['id'])
  modification_recipe_ingredients = modification_recipe['ingredients'].tolist()[0]

  ingredient_from_recipe = ['a']
  ingredient_from_KB = ['a']

  # picking up random ingredient from the chosen recipe and the KB to modify the recipe
  while ingredient_from_recipe[0]==ingredient_from_KB[0]:
    # print("hello")
    ingredient_from_recipe = random.sample(modification_recipe_ingredients,1)
    ingredient_from_KB = random.sample(KB,1)
  
  ingredient_from_recipe = ingredient_from_recipe[0]
  ingredient_from_KB = ingredient_from_KB[0]

  index = modification_recipe_ingredients.index(ingredient_from_recipe)

  # replacing the ingredient of the recipe with ingredient chosen from KB

  to_check = copy.deepcopy(modification_recipe_ingredients) 
  to_check[index] = ingredient_from_KB
  return_list = [to_check, modification_recipe_id, ingredient_from_KB]
  return return_list

In [109]:
for i in range(50):
  while 0<1:
    l = replace_ingredient()
    if is_same(l[0])==False:
      row_number = recipes[recipes['id'] == l[1]].index[0]
      cuisine = recipes.loc[row_number,'cuisine']
      to_append = pd.DataFrame({'id':[number_of_rows+1], 'cuisine':[cuisine], 'ingredients':[l[0]]})
      number_of_rows = number_of_rows+1
      recipes = recipes.append(to_append)
      KB.remove(l[2])
      break
  i = i+1